In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sympy as sp

In [259]:
# The times used for training (those in the loss function)

t_i, T_i = 0, 3

In [260]:
#Import the Underlying price dataset 
csv_file = 'Security_adj.csv'

#Use pandas to read the CSV file into a DataFrame
df = pd.read_csv(csv_file)

# Remove the first column and extract the first 80000 rows for train
x_train = df.iloc[:70016, 1:]

y_train = x_train.iloc[:, t_i].to_numpy()

# Print the first few rows of the DataFrame
print(x_train.head())

     0    1    2    3    4    5    6    7    8    9  ...  86  87  88  89  90  \
0  598  606  619  636  649  655  648  644  652  638  ...   0   0   0   0   0   
1  655  648  644  652  638    0    0    0    0    0  ...   0   0   0   0   0   
2  649  648  655  633  641  650  654  644  665  678  ...   0   0   0   0   0   
3  672  656  634  626  639  664  662  665  664  655  ...   0   0   0   0   0   
4  681  685  694  696  704    0    0    0    0    0  ...   0   0   0   0   0   

   91  92  93  94  95  
0   0   0   0   0   0  
1   0   0   0   0   0  
2   0   0   0   0   0  
3   0   0   0   0   0  
4   0   0   0   0   0  

[5 rows x 96 columns]


In [261]:
#Import the Underlying price dataset 
csv_file = 'Strike_adj.csv'

#Use pandas to read the CSV file into a DataFrame
df = pd.read_csv(csv_file)

# Remove the first column and extract the first 80000 rows for train
Strike_train = df.iloc[:70016, 1:]

y_train = np.vstack([y_train, Strike_train.iloc[:, t_i].to_numpy()])       #This will be used to compute the intrinsic value of the option

# Print the first few rows of the DataFrame
print(Strike_train.head())

     0    1    2    3    4    5    6    7    8    9  ...  86  87  88  89  90  \
0  600  600  600  600  600  600  600  600  600  600  ...   0   0   0   0   0   
1  655  655  655  655  655    0    0    0    0    0  ...   0   0   0   0   0   
2  550  550  550  550  550  550  550  550  550  550  ...   0   0   0   0   0   
3  710  710  710  710  710  710  710  710  710  710  ...   0   0   0   0   0   
4  630  630  630  630  630    0    0    0    0    0  ...   0   0   0   0   0   

   91  92  93  94  95  
0   0   0   0   0   0  
1   0   0   0   0   0  
2   0   0   0   0   0  
3   0   0   0   0   0  
4   0   0   0   0   0  

[5 rows x 96 columns]


In [262]:
#Import the Price dataset 
csv_file = 'Price_adj.csv'

#Use pandas to read the CSV file into a DataFrame
df = pd.read_csv(csv_file)

# Remove the first column
Price_train = df.iloc[:70016, 1:]

y_train = np.vstack([y_train, Price_train.iloc[:, t_i].to_numpy()])
y_train = y_train.T

# Print the first few rows of the DataFrame
print(Price_train.head())

    0   1   2   3   4   5   6   7   8   9  ...  86  87  88  89  90  91  92  \
0  14  16  25  38  52  56  51  44  51  37  ...   0   0   0   0   0   0   0   
1   9   9  14   8  17   0   0   0   0   0  ...   0   0   0   0   0   0   0   
2   9  10   8  11   9   8   8   9   6   5  ...   0   0   0   0   0   0   0   
3  37  49  71  78  67  44  46  44  44  53  ...   0   0   0   0   0   0   0   
4  54  57  65  66  74   0   0   0   0   0  ...   0   0   0   0   0   0   0   

   93  94  95  
0   0   0   0  
1   0   0   0  
2   0   0   0  
3   0   0   0  
4   0   0   0  

[5 rows x 96 columns]


In [263]:
y_train

array([[ 598,  600,   14],
       [ 655,  655,    9],
       [ 649,  550,    9],
       ...,
       [1888, 1910,    9],
       [1986, 1925,  135],
       [ 896, 1450,    0]], dtype=int64)

In [439]:
#Definire la loss function, quindi sarà necessario:
#Simulare le traiettorie del GBM
#Simulare le variabili Gaussiane

mini_batch = 64

frequencies = [1,2,3,4,5]   #REMEMBER TO THEN ADD THE COSINES FREQUENCIES


#FIRST IMPLEMENTATION WITH NO k, FOR EACH t_i WE HAVE ONLY ONE T_i UP TO NOW, ALSO WE ARE TRYING WITH ONLY ONE TIME

times = [t_i]             #the t_i
maturities = [T_i]        #the T_i


def quad_var_calc(freq, eval_time, horizon):
    # Function that evaluates the integral of sin^2(...), that is our single frequency variance
    result = eval_time/2 - (horizon*np.sin((4*freq*np.pi*eval_time)/horizon))/(8*freq*np.pi)

    return result


# In order to make TensorFlow able to correctly convert a function that creates a state (thus, that uses Variables)
# we have to break the function scope, declaring the variables outside of the function. This because we don't want
# to work in eager mode, which will cost us more time for the training
result = None

@tf.function
def GBM(mu = 1, sigma = 1, n = 50, dt = 0.001, x0 = 100, batch_size = 64):

    #np.random.seed(1)
    global result
    if result is None:
        result = tf.Variable(tf.zeros(shape=[batch_size], dtype=tf.dtypes.float64))
    for i in range(batch_size):
        x = np.exp(
            (mu - sigma ** 2 / 2) * dt
            + sigma * np.random.normal(0, np.sqrt(dt), size=(1, n)).T
        )
        x = np.vstack([1, x])
        #x = np.concatenate(x, axis=0)
        partial = x.cumprod(axis=0) 
        partial = x0[i] * partial[-1]     #return only last point of the GBM
        result[i].assign(partial)
    #tf.print(result)
    
    return result

def Martingale(frequencies, weights, eval_time = 1, horizon = 10, batch_size = 64):

    #Calculate the integral of the variance function m(t) in order to obtain the variance
    variance = 0
    quad_var = np.zeros(len(frequencies)*batch_size).reshape(batch_size,len(frequencies))

    #we calculate separately the variancies and multiply them by the weights tensor, then we'll sum the frequencies (in the Q_loss code)
    for i in range(len(frequencies)):
        for j in range(batch_size):
            quad_var[j][i] = quad_var_calc(frequencies[i], eval_time, horizon)
    #print(quad_var)

    variance = tf.keras.layers.Multiply()([weights,quad_var.reshape(batch_size, len(frequencies))])
    #tf.print(variance)

    # Define a Normal distribution with tensorflow
    random_value = tf.random.normal([batch_size,len(frequencies)], mean=0.0, stddev=tf.sqrt(variance), dtype=tf.dtypes.float32)
    #tf.print(random_value)
    return random_value


class Q_Loss(tf.keras.losses.Loss):
    def __init__(self, times, maturity, lambd = 0, name="Q_loss", **kwargs):
        super().__init__(name=name, **kwargs)
        self.times = times
        self.maturity = maturity
        self.lamb = lambd

    def call(self, y_true, y_pred):
        #These are arrays of lenght = self.times and size = mini_batch with all the x_i, strike_i and g_i needed
        ind = tf.constant([0])
        x = tf.transpose(tf.nn.embedding_lookup(tf.transpose(y_true), ind))
        #tf.print(x)
        ind = tf.constant([1])
        strike = tf.transpose(tf.nn.embedding_lookup(tf.transpose(y_true), ind))

        
        ind = tf.constant([2])
        g = tf.transpose(tf.nn.embedding_lookup(tf.transpose(y_true), ind))
        #tf.print(strike)
        # convert them to float64
        x = tf.cast(x, tf.dtypes.float64)
        strike = tf.cast(strike, tf.dtypes.float64)
        g = tf.cast(g, tf.dtypes.float64)

        # Reshape from (64,1) to (64,)
        strike = tf.reshape(strike, (mini_batch,))


        #for i in range(len(self.times)):       #We'll have to update in order to consider more t_i than just one

        # compute the underlying quantity
        x_star = tf.math.log(GBM(mu = 1, sigma = 1, n = self.maturity[0], dt = 0.001, x0 = x, batch_size = mini_batch))
        x_star = tf.cast(x_star, tf.dtypes.float64)
        
        mart = Martingale(frequencies, y_pred, self.maturity[0], 96, mini_batch)
        mart = tf.math.reduce_sum(mart,1)       #Sum all the frequencies together
        mart = tf.cast(mart, tf.dtypes.float64)
        #tf.print(x_star)

        # We try as G_i,k the option intrinsic value
        loss = tf.square(tf.math.maximum(tf.exp(x_star + mart)-strike, tf.constant([0], dtype=tf.float64)) - g)     #+ self.lambd*Martingale(variance = y_pred)  # we'll have to add the regularization term

        return tf.math.reduce_mean(loss)

    def get_config(self):
        config = {
            'times': self.times,
            'maturity': self.maturity,
            'lambda': self.lambd
        }
        base_config = super().get_config()
        return {**base_config, **config}

In [440]:
#Definire il modello, ci saranno 3 steps:
#1) Solo Layer di ReLU
#2) Layer di ReLU + sin
#3) Provare le fancy activation functions

# Create a simple model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='sigmoid', input_shape=(96,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='sigmoid'),
    tf.keras.layers.Dense(5, activation='softmax')  # Softmax activation for probability distribution
])

In [441]:
#Trainare il modello sul dataset e testarlo

loss_fn = Q_Loss(times, maturities)

# Compile the model with our personalized loss function

# when we'll have a working loss function we'll have to remove the run_eagerly and rewrite properly the GBM function
# since it is used in order to convert x0 to numpy and then convert it back, but we can do this all only with tensors
# speeding up a lot the calculations
model.compile(optimizer='adam', loss=loss_fn, run_eagerly=False)

# Train the model with your personalized loss function
model.fit(x_train, y_train, epochs=5, batch_size=mini_batch)

Epoch 1/10
1094/1094 [==============================] - 5s 4ms/step - loss: 194519.0156
Epoch 2/10
1094/1094 [==============================] - 4s 4ms/step - loss: 183332.0625
Epoch 3/10
1094/1094 [==============================] - 4s 3ms/step - loss: 179248.8125
Epoch 4/10
1094/1094 [==============================] - 4s 4ms/step - loss: 180974.0625
Epoch 5/10
1094/1094 [==============================] - 4s 4ms/step - loss: 180842.5781
Epoch 6/10
1094/1094 [==============================] - 4s 4ms/step - loss: 182397.8125
Epoch 7/10
1094/1094 [==============================] - 4s 4ms/step - loss: 179949.1562
Epoch 8/10
1094/1094 [==============================] - 4s 4ms/step - loss: 181112.2500
Epoch 9/10
1094/1094 [==============================] - 4s 4ms/step - loss: 181040.3750
Epoch 10/10
1094/1094 [==============================] - 4s 4ms/step - loss: 181467.7656


In [442]:
predictions = model.predict(x_train)

2188/2188 [==============================] - 2s 775us/step


In [443]:
tensor_pred = tf.convert_to_tensor(predictions[0])

In [444]:
tensor_pred = tf.reshape(tensor_pred, (1,5))
tensor_pred

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[9.9999642e-01, 1.2711969e-06, 7.7364047e-07, 7.1407646e-07,
        8.2845776e-07]], dtype=float32)>

In [445]:
tf.convert_to_tensor([x_train.iloc[0,0]], dtype = tf.float64)

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([598.])>

In [446]:
x_star = tf.math.log(GBM(mu = 1, sigma = 1, n = 2, dt = 0.001, x0 = tf.convert_to_tensor([x_train.iloc[0,0]], dtype = tf.float64), batch_size = 1))

In [460]:
# delta between predicted underlying with GBM and actual value

np.exp(x_star[0].numpy())- x_train.iloc[0,2]

-25.91629691108608

In [502]:
mart = Martingale(frequencies, tensor_pred, 2, 96, 1)

# delta between predicted underlying with GBM + martingale and actual value

np.exp(x_star[0].numpy() + tf.math.reduce_sum(mart,1).numpy()) - x_train.iloc[0,2]

array([-17.053284], dtype=float32)